## ETA

In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('./product_data/match_results/match_result.csv')
len(data)

1063483

In [ ]:
import requests
import re
session = requests.Session()
print("————————————————————————————————————————————————————")
print("—        悟空天气网支持查询 经纬度 对应天气信息          —")
print("————————————————————————————————————————————————————")
 
 
def get_weather_info(lon, lat):
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,'
                  'image/avif,image/webp,image/apng,*/*;q=0.8,application/'
                  'signed-exchange;v=b3;q=0.7',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Cache-Control': 'max-age=0',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Cookie': f"lon={lon}; lat={lat}",
        'Origin': 'https://wuknet.com',
        'Referer': 'https://wuknet.com/weather',
        'Sec-Ch-Ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': 'macOS',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) '
                     'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
        }
 
    target_url = 'https://wuknet.com/weather'
    response = requests.post(url=target_url, headers=headers).text
    #print(response)
    ex_weather = r'<td>实时：<b>(.*?)</b>'
    ex_sense_tem = r'">感知度：(.*?)&.*?</span>'
 
    ex_highest_tem = r'<span class="wable">最高温：(.*?) &.*?</span>'
    ex_lowest_tem = r'<span class="wable">最低温：(.*?) &.*?</span>'
    ex_humidity = r'<span class="wable">湿度：(.*?)</span>'
 
    ex_pressure = r'<span class="wable">大气压力：(.*?)&.*?</span>'
    ex_cloud = r'<span class="wable">云量：(.*?)</span>'
 
    ex_atmosphere_tem = r'<span class="wable">大气温度：(.*?)&.*?</span>'
 
    ex_wind_speed = r'<span class="wable">风速：(.*?)</span>'
    ex_wind_direction = r'<span class="wable">风向：(.*?)</span>'
 
    ex_ultraviolet = r'<span class="wable">紫外线指数(.*?)</span>'
    ex_visibility = r'<span class="wable">平均能见度：(.*?)</span>'
 
    ex_sun_up = r'<span class="wable">日出时间：(.*?)</span>'
    ex_sun_down = r'<span class="wable">日落时间：(.*?)</span>'
 
    weather = re.findall(ex_weather, response, re.S)
    sense_tem = re.findall(ex_sense_tem, response, re.S)
 
    highest_tem = re.findall(ex_highest_tem, response, re.S)
    lowest_tem = re.findall(ex_lowest_tem, response, re.S)
    humidity = re.findall(ex_humidity, response, re.S)
 
    pressure = re.findall(ex_pressure, response, re.S)
    cloud = re.findall(ex_cloud, response, re.S)
 
    atmosphere_tem = re.findall(ex_atmosphere_tem, response, re.S)
 
    wind_speed = re.findall(ex_wind_speed, response, re.S)
    win_direction = re.findall(ex_wind_direction, response, re.S)
 
    ultraviolet = re.findall(ex_ultraviolet, response, re.S)
    visibility = re.findall(ex_visibility, response, re.S)
 
    sun_up = re.findall(ex_sun_up, response, re.S)
    sun_down = re.findall(ex_sun_down, response, re.S)
    print(f"\n为您查询到「经度：{lon}-纬度：{lat}」的如下天气信息")
    print(f"当前天气：{weather[0]}")
    print(f"体感温度：{sense_tem[0]}")
    print(f"最高气温：{highest_tem[0]}")
    print(f"最低气温：{lowest_tem[0]}")
    print(f"湿度：{humidity[0]}")
    print(f"大气压力：{pressure[0]}")
    print(f"云量：{cloud[0]}")
    print(f"大气温度：{atmosphere_tem[0]}")
    print(f"风速：{wind_speed[0]}")
    print(f"风向：{win_direction[0]}")
    print(f"紫外线指数：{ultraviolet[0]}")
    print(f"平均能见度：{visibility[0]}")
    print(f"日出时间：{sun_up[0]}")
    print(f"日落时间：{sun_down[0]}\n")
 
 
lon = 104.10242 # float(input("请输入经度（小数形式）:"))
lat = 30.662 # float(input("请输入纬度（小数形式）:"))
get_weather_info(lon, lat)

————————————————————————————————————————————————————
—        悟空天气网支持查询 经纬度 对应天气信息          —
————————————————————————————————————————————————————
